In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import shutil



DATASET FOLDER MAKING FUNCTIONS

In [ ]:
def load_words(pth : str) -> list[str]:
    words=[]
    with open(pth,'r',encoding='utf-8') as words_text:
        for line in words_text:
           indexed_words=line.strip().split('. ',1) #indexed_words is raw data with number
           word=indexed_words[1]          #separating number and words
           words.append(word)
    return words # returns a useable words list from text file

In [ ]:
def make_folders(location:str,prt_list,chld_list):
    for folder in prt_list:
        try:
            if not os.path.exists(os.path.join(location,str(folder))):
                os.mkdir(os.path.join(location,str(folder)))
            else:
                print(f"parent foldar already exist{os.path.join(location,str(folder))}")


            for ch_folder in chld_list:
                    
                try:
                    if not os.path.exists(os.path.join(location,str(folder),str(ch_folder))):
                        os.mkdir(os.path.join(location,str(folder),str(ch_folder)))
                    else:
                        print(f"child foldar already exist{os.path.join(location,str(folder),str(ch_folder))}")        
                except:
                    print(f"invalid syntax in child{os.path.join(location,str(folder),str(ch_folder))}")         

        except:
            print(f"invalid syntax in parent{os.path.join(location,str(folder))}")
        
            


                                                    #for making dataset folders
                                                     commands and variables :  
                                                                 total funtion  = 2
                                                                 variable required= 3

In [ ]:
total_speaker=200
word_text_path="uniquelist.txt" # path for file that contains words list
datast_path='.\Dataset' #path where we want to save the data set

parent_folders=[f"s{i}" for i in range(1,total_speaker+1)] #generating parent folder ie. speaker folder s1,s2,s3,s4

#make_folders(datast_path,parent_folders,load_words('uniquelist.txt'))


XL     FINDING,    READING (FINDING USABLE VIDS)     FUNCTIONS

In [ ]:
# grabs path of files in base location #base means folders where vdos and xls are stored
def parent():
    parent=os.listdir(base_loc)
    parent=[os.path.join(base_loc,file) for file in parent]
    return parent

In [ ]:
#grabbing xl from the location
def find_xl(pth):
    all_files=os.listdir(pth)
    xl = [file for file in all_files if file.endswith('.xlsx')]
    xl_loc=os.path.join(pth,xl[0])
    try:
        #print(xl)
        if len(xl)==1:
            return xl_loc
        else:
            print("Too many xl")
            xl=[]
    except:
        print('no xl error')

In [ ]:
def read_xl_vids(poth:str) -> list[str]:  #returns annotated vdo list from xl
    df=pd.read_excel(poth)
    col=[i for i in df.columns]
    col=col[2:]
    annotated_vids=[i for i in col if (df[i] == "0::.,0::." ).sum()<50]
    try:
        return annotated_vids
    except:
        print("no particular annotation")

COMMAND FOR FINDING USEABLE VIDEO FROM BASE DIRECTORY OF FOLDERS

In [ ]:
base_loc='.\demo'
parent()[0]

In [ ]:
find_xl(parent()[0])

In [ ]:
base_loc='.\demo' # location (parent folder) of vdos and xls

read_xl_vids(find_xl(parent()[1]))

VIDEO TimeString to Miliseconds,CHOPPING FUNCTIONS


In [ ]:
def tstr2ms(time_string):
    # Split the time string into hours, minutes, seconds, and milliseconds
    time_parts = time_string.split(':')

    # Convert hours, minutes, seconds, and milliseconds to integers (use 0 if missing)
    hours = int(time_parts[0]) if len(time_parts) > 0 and time_parts[1]!="" else 0
    minutes = int(time_parts[1]) if len(time_parts) > 1 and time_parts[1]!="" else 0
    seconds_parts = time_parts[2].split('.')
    seconds = int(seconds_parts[0]) if len(seconds_parts)> 0 and seconds_parts[0]!=""  else 0
    if len(str(seconds_parts[1]))<2:
           milliseconds = (int(seconds_parts[1])*100)  if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
           #print("in the one digit range")
    elif len(str(seconds_parts[1]))<3:
         milliseconds = (int(seconds_parts[1])*10)  if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
         #print("in the two digit range")
    else:
        milliseconds = int(seconds_parts[1]) if len(seconds_parts) > 1 and seconds_parts[1]!="" else 0
        #print("in the three digit range")



    # Calculate total milliseconds
    total_milliseconds = (
        hours * 60 * 60 * 1000 +
        minutes * 60 * 1000 +
        seconds * 1000 +
        milliseconds
    )

    return total_milliseconds




In [ ]:
'''
import time
import glob
buff=[]

def save_frames(video_path, output_folder, start_time, end_time):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get frames per second (fps)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate start and end frame indices
    start_frame = int(start_time/1000 * fps)
    end_frame = int(end_time/1000 * fps)

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Set the video capture position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Read and save frames within the specified range
    frame_count = start_frame
    i=1
    _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]
    while frame_count <= end_frame:
        ret, frame = cap.read()
        #get the width and height
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        if not ret:
            print(f"Error: Could not read frame {frame_count}.")
            break

        # Save the frame as an image in the output folder
        frame_filename = os.path.join(r"D:\Dataset\buffer", f"{i:02d}.png")
        if width!=1280 and height!=720:
            frame=cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_AREA)
        try:
            
          if not os.listdir(r"D:\Dataset\buffer"):
            cv2.imwrite(frame_filename,frame,[cv2.IMWRITE_PNG_COMPRESSION, 5])
            time.sleep(.01)
            print("image has been written in buffer folder")
            
            try:
             if not os.path.exists(os.path.join(output_folder,f"{i:02d}.png")):
                shutil.move(frame_filename,output_folder)
                print(f"succesfully moved to{output_folder}")
             else:
                 print(f'file already exists in {output_folder}')
                 _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]
                 print('Buffer cleared')
            except:
             print("couldn't move to folder")
             _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]
             print('Buffer cleared')
            
          else:
             print("file already available in buffer")
             buff.append(output_folder)
             _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]
             print('Buffer cleared')
        except:
            print("openCV error :couldn't save image in buffer")
            buff.append(output_folder)
       
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"width={width},height={height}") 
        

        frame_count += 1
        i+=1
    
    # Release the video capture object
    cap.release()
    return print(buff)
    
    '''

In [ ]:
import cv2
import os
import shutil
import glob
import concurrent.futures

def save_frame(frame_info):
    frame_count, frame, frame_filename, output_folder = frame_info
    cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_PNG_COMPRESSION, 5])
    shutil.move(frame_filename, output_folder)
    
    print(f"Frame {frame_count} saved to {output_folder}")
    

def save_frames(video_path, output_folder, start_time, end_time):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get frames per second (fps)
    fps = cap.get(cv2.CAP_PROP_FPS)
    # Calculate start and end frame indices
    start_frame = int(start_time/1000 * fps)
    end_frame = int(end_time/1000 * fps)

    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Set the video capture position to the start frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Read and save frames within the specified range
    frame_count = start_frame
    i = 1
    _ = [os.remove(file) for file in glob.glob(os.path.join(r"D:\Dataset\buffer", '*.png'))]

    frames_info = []

    while frame_count <= end_frame:
        ret, frame = cap.read()
        #get the width and height
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        if not ret:
            print(f"Error: Could not read frame {frame_count}.")
            break

        # Save the frame as an image in the output folder
        frame_filename = os.path.join(r"D:\Dataset\buffer", f"{i:02d}.png")
        if width != 1280 and height != 720:
            frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_AREA)

        frames_info.append((frame_count, frame.copy(), frame_filename, output_folder))  # Use copy to avoid shared memory issues

        frame_count += 1
        i += 1

    # Release the video capture object
    cap.release()

    # Use ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(save_frame, frames_info)

    return




In [ ]:
def chk_annotation(vdi:str)->list[int]:
    annt=[a for a in df[vdi]]
    #print(annt)
    stym=[]
    etym=[]
    
    for k,val in enumerate(annt):
        try:
         stym.append(tstr2ms(val.split(",")[0])) 
         etym.append(tstr2ms(val.split(",")[1]))
         #stym.append(val.split(",")[0]) #uncomment for movie pie
         #etym.append(val.split(",")[1])
         if tstr2ms(val.split(",")[0])>tstr2ms(val.split(",")[1]):
            print(f"error:STRT-TIME > END-TIME in index {k+1}")
         if abs(tstr2ms(val.split(",")[0])-tstr2ms(val.split(",")[1]))<200:
            print(f"error:less than 200ms for word in index {k+1}")
         if abs(tstr2ms(val.split(",")[0])-tstr2ms(val.split(",")[1]))>3000:
            print(f"error:more than 3 sec for word in index {k+1}")
        
        except:
           print(f"error:format doesn't match in index {k+1}")   
      
    return stym,etym
    


STEP1: Find XL location and read the XL

STEP2: Find Annotated vdos from the XL


STEP3: Check the annotations


STEP4: Make input and output folders


RUN THE BOX



In [ ]:
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import shutil
import concurrent.futures
import time

In [ ]:
i=0 #total number of folder inside demo to iterate put value of i accordingly
base_loc='.\demo' # location (parent folder) of vdos and xls

xl_path=find_xl(parent()[i])
df=pd.read_excel(xl_path)

read_xl_vids(find_xl(parent()[i]))
print(f"this is {parent()[i]} and the xl is {find_xl(parent()[i])} annotated vids are {read_xl_vids(find_xl(parent()[i]))}")


In [ ]:
vid_name="vid_7.mp4" 

In [ ]:
df=pd.read_excel(xl_path)
chk_annotation(vid_name)

In [ ]:

stym,etym=chk_annotation(vid_name)
print(stym)
print(etym)


In [ ]:

video_path=os.path.join(parent()[i],vid_name)  #videos from base location
print(video_path)

output_path=r'.\Dataset'


In [ ]:
######################## Change it with videos
speaker_folder='s1' 

In [ ]:

for i,word in enumerate(df['words']):
    output_folder=os.path.join(output_path,speaker_folder,word)
    start_time=stym[i]
    end_time=etym[i]
    if not start_time==end_time:
       save_frames(video_path, output_folder, start_time, end_time)
    


        
    

